In [1]:
%matplotlib inline

# Imports

In [1]:
# Author: Till Zemann
# License: MIT License

from __future__ import annotations

from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib.patches import Patch
from tqdm import tqdm

import gymnasium as gym
import gym_trading_env as gtenv
from gym_trading_env.downloader import download
import datetime


# Download training data

In [12]:
url = "https://raw.githubusercontent.com/ClementPerroud/Gym-Trading-Env/main/examples/data/BTC_USD-Hourly.csv"
df = pd.read_csv(url, parse_dates=["date"], index_col= "date")
df.sort_index(inplace= True)
df.dropna(inplace= True)
df.drop_duplicates(inplace=True)
df.head()

,unix,symbol,open,high,low,close,volume,Volume USD
date,,,,,,,,
2018-05-15 06:00:00,1526364000,BTC/USD,8733.86,8796.68,8707.28,8740.99,4906603.14,559.93
2018-05-15 07:00:00,1526367600,BTC/USD,8740.99,8766.00,8721.11,8739.00,2390398.89,273.58
2018-05-15 08:00:00,1526371200,BTC/USD,8739.00,8750.27,8660.53,8728.49,7986062.84,917.79
2018-05-15 09:00:00,1526374800,BTC/USD,8728.49,8754.40,8701.35,8708.32,1593991.98,182.62
2018-05-15 10:00:00,1526378400,BTC/USD,8708.32,8865.00,8695.11,8795.90,11101273.74,1260.69


In [9]:
# Download data from 2017-01-01 to 2022-12-31
# Download BTC/USDT historical data from Binance and stores it to directory ./data/binance-BTCUSDT-1h.pkl
download(exchange_names = ["binance"],
    symbols= ["BTC/USDT"],
    timeframe= "1h",
    dir = "data",
    since = datetime.datetime(year= 2017, month= 1, day=1),
)
# Import your fresh data
df = pd.read_pickle("./data/binance-BTCUSDT-1h.pkl")
df.head()

BTC/USDT downloaded from binance and stored at data/binance-BTCUSDT-1h.pkl


,open,high,low,close,volume,date_close
date_open,,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17 05:00:00
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17 06:00:00
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17 07:00:00
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17 08:00:00
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 09:00:00


In [14]:
df = pd.read_pickle('./data/binance-BTCUSDT-1h.pkl')
trainingDF = df.truncate(
    after = pd.Timestamp('2023-01-01'),
    copy = True
)

trainingDF.dropna(inplace=True)
trainingDF.sort_index(inplace=True)
trainingDF.to_csv('./data/binance_training_data.csv')

In [13]:
df = pd.read_pickle('./data/binance-BTCUSDT-1h.pkl')
testingDF = df.truncate(
    before = pd.Timestamp('2023-01-01'),
    copy = True
)

testingDF.dropna(inplace=True)
testingDF.sort_index(inplace=True)
testingDF.to_csv('./data/binance_testing_data.csv')

# Create features

- These are the state vector inputs
- this would be an import from `TAindicators`
- only data columns starting with "`feature_`" will appear in the observation space

In [13]:
# df is a DataFrame with columns : "open", "high", "low", "close", "Volume USD"

# Create the feature : ( close[t] - close[t-1] )/ close[t-1]
df["feature_close"] = df["close"].pct_change()

# Create the feature : open[t] / close[t]
df["feature_open"] = df["open"]/df["close"]

# Create the feature : high[t] / close[t]
df["feature_high"] = df["high"]/df["close"]

# Create the feature : low[t] / close[t]
df["feature_low"] = df["low"]/df["close"]

 # Create the feature : volume[t] / max(*volume[t-7*24:t+1])
df["feature_volume"] = df["Volume USD"] / df["Volume USD"].rolling(7*24).max()

df.dropna(inplace= True) # Clean again !
# Eatch step, the environment will return 5 inputs  : "feature_close", "feature_open", "feature_high", "feature_low", "feature_volume"

# Define custom rewards

In [ ]:
def lr(history: dict):
    """logarithmic return"""
    if history['position', -1] == 0:
        return 0
    else:
        return np.log(history['data_close', -1]) - np.log(history['data_close', -2])
    
def alr(history: dict):
    """average logarithmic return"""
    pass

def sr(history):
    """sharpe ratio"""
    pass

def powc(history):
    """sharpr ratio"""

def reward_function(history):
    pass

# Create Environment

In [15]:
env = gym.make("TradingEnv",
        name= "BTCUSD",
        df = df, # Your dataset with your custom features
        positions = [0, 1], # -1 (=SHORT), 0(=SELL ALL), +1 (=BUY ALL)
        trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
        borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
    )

# Define Agent

In [ ]:
class BitcoinTrainingAgent:
    def __init__(self) -> None:
        """Initialize hyperparameters"""
        pass

    def get_action(self, observation):
        """Given an observation, choose an action"""
        pass

    def update(self):
        """Update parameters"""
        pass

# Create Agent

In [ ]:
# TODO: define hyperparameters and pass to agent
bitcoinAgent = BitcoinTrainingAgent()

# Train model using agent

In [16]:
# Run an episode until it ends :
done, truncated = False, False
observation, info = env.reset()
while not done and not truncated:
    # Pick a position by its index in your position list (=[-1, 0, 1])....usually something like : position_index = your_policy(observation)
    position_index = env.action_space.sample() # At every timestep, pick a random position index from your position list (=[-1, 0, 1])
    observation, reward, done, truncated, info = env.step(position_index)

Market Return : 423.10%   |   Portfolio Return : -94.73%   |   
